# Metoda algebraiczna
### Implementacja

In [1]:
# Metoda algebraiczna
# x jest macierzą opisująca równania
# C jest stałą reprezentującą szybkośc rozchodzenia sie fali (tutaj predkosc swiatla), podniesioną do kwadratu
#C = 0.047*0.047 # stała 
C = 1

setprecision(BigFloat, 256)

# zwaraca trójke (ilosc_rozwiazan, pierwsze, drugie)
function solve_quadratic(a, b, c)
    if (c == 0)
        # ax + b = 0, stad x = -b/a
        return (2, 0, -b/a)
    end
    
    # dalej zakladamy c != 0
    
    # moze byc problem przy 4ac bliskim b^2
    δ = b*b - 4*a*c
    
    if (δ < 0)
        return (0, 0, 0)
    elseif (δ == 0)
        return (1, -b/2a, -b/2a)
    else
        δ_s = sqrt(δ)
        
        # problem pojawia sie wtedy, gdy któreś z wyrażeń -b +- δ_s jest bliskie zeru
        if (b < 0)
            sign = 1
        else
            sign =-1
        end

        x1 = (-b + sign * δ_s) / (2.0*a)
        x2 = c/(x1*a)
        
        return (2, x1, x2)
    end
end

function vector_length(x::Array{BigFloat, 1})
    l = length(x)
    sum = 0
    for i = 1:l
        sum = sum + x[i]*x[i]
    end
    
    return sqrt(sum)
end

# Wynik w postaci (x, y, z, T)
# parametr returnAdditional ustawiony na true zwraca nam również drugie, "fałszywe" rozwiązanie
function solve(x::Array{BigFloat, 2}, returnAdditional = false)
    # create matrix
    matrix = Array{BigFloat, 2}([
            2*(x[2, 1] - x[1, 1]) 2*(x[2, 2] - x[1, 2]) 2*(x[2, 3] - x[1, 3]);
            2*(x[3, 1] - x[1, 1]) 2*(x[3, 2] - x[1, 2]) 2*(x[3, 3] - x[1, 3]);
            2*(x[4, 1] - x[1, 1]) 2*(x[4, 2] - x[1, 2]) 2*(x[4, 3] - x[1, 3])
            ])
    
    s1 = x[1,1]*x[1,1] + x[1,2]*x[1,2] + x[1,3]*x[1,3]
    s2 = x[2,1]*x[2,1] + x[2,2]*x[2,2] + x[2,3]*x[2,3]
    s3 = x[3,1]*x[3,1] + x[3,2]*x[3,2] + x[3,3]*x[3,3]
    s4 = x[4,1]*x[4,1] + x[4,2]*x[4,2] + x[4,3]*x[4,3]
    
    solMatrix = Array{BigFloat, 2}([
            2*C*(x[2, 4] - x[1, 4]) C*(x[1,4]*x[1,4] - x[2,4]*x[2,4]) + s2 - s1;
            2*C*(x[3, 4] - x[1, 4]) C*(x[1,4]*x[1,4] - x[3,4]*x[3,4]) + s3 - s1;
            2*C*(x[4, 4] - x[1, 4]) C*(x[1,4]*x[1,4] - x[4,4]*x[4,4]) + s4 - s1
            ])
    
    y = matrix \ solMatrix
    
    r1 = x[1,1] - y[1,2] # x1 - cx indeksy dolne
    r2 = x[1,2] - y[2,2] # y1 - cy indeksy dolne
    r3 = x[1,3] - y[3,2] # z1 - cz indeksy dolne
    
    # wspolczynniki rownania kwadratowego, które otrzymujemy
    a = y[1,1]*y[1,1] + y[2,1]*y[2,1] + y[3,1]*y[3,1] - C
    b = 2*(-y[1,1]*r1 - y[2,1]*r2 - y[3,1]*r3 + C*x[1,4])
    c = r1*r1 + r2*r2 + r3*r3 - C*x[1,4]*x[1,4]
    
    # wynik dla T
    (solutions_number, T1, T2) = solve_quadratic(a, b, c)
    
    x1 = y[1, 2] + T1 * y[1, 1]
    y1 = y[2, 2] + T1 * y[2, 1]
    z1 = y[3, 2] + T1 * y[3, 1]
    
    x2 = y[1, 2] + T2 * y[1, 1]
    y2 = y[2, 2] + T2 * y[2, 1]
    z2 = y[3, 2] + T2 * y[3, 1]
    
    # Jeśli ustalono dodatkowo parametr returnAddtional, wtedy zwracamy oba rozwiązania
    # nie uwzględniając, które z nich jest poprawne w rzeczywistości
    if (returnAdditional)
        return (x1, y1, z1, T1, x2, y2, z2, T2)
    end
    
    # obliczamy długość wektorów obu rozwiązań
    l1 = vector_length([x1, y1, z1])
    l2 = vector_length([x2, y2, z2])
    
    # szukamy wektora najbliższego "jedynce", bo taki punkt leży na powierzchni ziemi.
    if (abs(l1 - 1) < abs(l2 - 2))
        # bierzemy l1
        return (x1, y1, z1, T1)
    else
        # bierzemy l2
        return (x2, y2, z2, T2)
    end
end

solve (generic function with 2 methods)

# Testy

### Test porównywany z WolframAlpha

In [2]:
input = Array{BigFloat, 2}([
        234 123 342 354
        231 234 413 432
        321 352 643 642
        213 302 521 523
        ])

(x, y, z, T) = solve(input)

#wynik z wolframi dla input2
#www.wolframalpha.com/input/?i=(x-234)%5E2%2B(y-123)%5E2%2B(z-342)%5E2%3D(354-t)%5E2%26(x-231)%5E2%2B(y-234)%5E2%2B(z-413)%5E2%3D(432-t)%5E2%26(x-321)%5E2%2B(y-352)%5E2%2B(z-643)%5E2%3D(642-t)%5E2%26(x-213)%5E2%2B(y-302)%5E2%2B(z-521)%5E2%3D(523-t)%5E2
wt=BigFloat(258.90541253521979255169217359014046413298532297538496213700950084891090986079279748715260300885471628639428258935548708617074093045927643023059966474908285331652876368532838597086974632959399876248459798137375107041025588345608791602437461595225006236650415890733337325157320245930304311528397848893036872242961094526054)
wx=BigFloat(223.97605295001951333752434779788694572788843073598178637064134221807226910690887169329937636911868205554791171603978094083021244405995854257404538636105038891389729785876273152603743979230619944696330804570137409421282285131650441693455534368016310746434438468256083701709075569430631865349001181222157309587196803167973)
wy=BigFloat(61.04855730339634595739853477070692526098388632382039894935164939809653230848396618322247969975455103201107108181604331460909801324837418755964076023938653212800110907221228712080692660378951191317797066506111207616428761160172560969277675603957978602017641354021401860035385691274352592735656124370264428333344536079944)
wz=BigFloat(413.44592219605926701089190456163589205052548031470293130169904238986693929888513482732543545246857397115730826811642996650983873855540707558575754398506631914398327259177941450558286873330443210495681440651370796225919017712299757781417839654820616815127853022332295154978456255766232311290200083834140794629736792643132)

@show x
@show wx
@show y
@show wy
@show z
@show wz
@show T
@show wt

@show BigFloat(x - wx)
@show BigFloat(y - wy)
@show BigFloat(z - wz)
@show BigFloat(T - wt)

x = 2.239760529500195133375243477978869457278884307359817863706413422180722691069268e+02
wx = 2.239760529500195218588487477973103523254394531250000000000000000000000000000000e+02
y = 6.104855730339634595739853477070692526098388632382039894935164939809653230848702e+01
wy = 6.104855730339634334313814179040491580963134765625000000000000000000000000000000e+01
z = 4.134459221960592670108919045616358920505254803147029313016990423898669392988874e+02
wz = 4.134459221960592572031600866466760635375976562500000000000000000000000000000000e+02
T = 2.589054125352197925516921735901404641329853229753849621370095008489109098608017e+02
wt = 2.589054125352197956999589223414659500122070312500000000000000000000000000000000e+02
BigFloat(x - wx) = -8.521324399999423406597551022389018213629358657781927730893073176666423964557241e-15
BigFloat(y - wy) = 2.614260392980302009451352538667570398949351649398096532308487023993106772487936e-15
BigFloat(z - wz) = 9.807731817914959828512927824064702931301699042389866939

-3.148266748751325485879221708274615037862990499151089090139198330751267849821542e-15

## Losowo wygenerowane testy

In [4]:
function gen_test(satellites)
    # wektor [x; y; z; t] - (x, y, z): nasza pozycja, t: błąd zegara
    X = vcat(Array{BigFloat}(2000 * rand(3) - 1000) .^ 3, Array{BigFloat}(100 * randn(1)) .^ 3)
    data = Array{BigFloat, 2}(4, satellites)
    for i = 1:satellites
        v = Array{BigFloat}(2000 * rand(3) - 1000) .^ 3
        data[:, i] = X + vcat(v, norm(v) / C)
    end
    return data, X
end

function transposeMatrix(matrix)
    tmatrix = Array{BigFloat, 2}(4, 4)
    
    for i = 1:4
        for j = 1:4
            tmatrix[i, j] = matrix[j, i]
        end
    end
    
    return tmatrix
end

function test(testsNumber)
    maxError = 0
    for i = 1:testsNumber
        (X, data) = gen_test(4)
        transposedX = transposeMatrix(X)
        (x1, y1, z1, T1, x2, y2, z2, T2) = solve(transposedX, true)
        
        # testy są generowane losowo, zatem potrzebujemy dwoch rozwiazań z metody solve, aby sprawdzić dokładność wyników
        x = x1
        y = y1
        z = z1
        T = T1
        if (abs(T1 - data[4]) > abs(T2 - data[4]))
             x = x2
             y = y2
             z = z2
             T = T2
        end
        
        epsX = abs(x - data[1])
        epsY = abs(y - data[2])
        epsZ = abs(z - data[3])
        epsT = abs(T - data[4])
        
        maxLocalError = max(epsX, epsY, epsZ, epsT)
        if (maxLocalError > maxError)
            maxError = maxLocalError
        end
    end
    
    return maxError
end

test (generic function with 1 method)

In [11]:
test(10000000)

8.448795652460897917462846907393738747436583937124256148925268525152959727843252e-57

In [5]:
test(100000000)

4.683706264871538273135380579580361381328516195490738968722945346639215112792924e-54